## Set up the run parameters parameters

In [ ]:
#use these parameters for testing this notebook outside of the automated loop of q1k_automated_reports.ipynb
subject_id_in = "100134_F1"
subject_id_out = "100134F1"
task_id_in = "VEP"
task_id_in_et = "VEP" 
task_id_out = "VEP"
run_id = "1"
session_id = "01"
project_path = "/project/def-emayada/q1k/experimental/HSJ/"
dataset_group = "experimental"
site_code = "HSJ" #'MHC' or 'HSJ'
et_sync = True

#use these empty parameters when executing this notebook from an automation script.
#subject_id_in = ""
#subject_id_out = ""
#task_id_in = ""
#task_id_in_et = "" 
#task_id_out = ""
#run_id = ""
#session_id = ""
#project_path = ""
#dataset_group = ""
#site_code = ""
#et_sync = True

print('subject_id_in: ' + subject_id_in)
print('subject_id_out: ' + subject_id_out)
print('task_id_in: ' + task_id_in)
print('task_id_in_et: ' + task_id_in_et)
print('task_id_out: ' + task_id_out)
print('run_id: ' + run_id)
print('session_id: ' + session_id)
print('project_path: ' + project_path)
print('dataset_group: ' + dataset_group)
print('site_code: ' + site_code)


In [ ]:
# import packages
import pandas as pd
import numpy as np
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"
import q1k_init_tools as qit
import glob
import re
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns

In [ ]:
#define the DIN events associated with events of interest based on the task_id..
din_str, event_dict_offset = qit.set_din_str(task_id_out)

## Read the EEG file

In [ ]:
subject_id_in

In [ ]:
# generate the input paths
session_file_name_eeg, session_file_name_et = qit.generate_session_ids(dataset_group, project_path, site_code, task_id_in, subject_id_in, run_id)

In [ ]:
session_file_name_eeg[0]

In [ ]:
# read the input EEG session
print('Reading: ' + session_file_name_eeg[0])
eeg_raw = mne.io.read_raw_egi(session_file_name_eeg[0])
eeg_raw_fresh=eeg_raw.copy() #make a fresh copy for later
device_info=eeg_raw.info['device_info']

In [ ]:
# peak... at the EEG channel types
channel_types = eeg_raw.get_channel_types()
print("EEG Channel Types:", channel_types)
print("EEG Channel Names:", eeg_raw.info['ch_names'])


In [ ]:
# show the sensor topography
fig = eeg_raw.plot_sensors(show_names=True)

## Get and modify the EEG event structures

In [ ]:
# create the EEG event structures
eeg_events = mne.find_events(eeg_raw, shortest_event = 1)
eeg_event_dict = qit.get_event_dict(eeg_raw, eeg_events, event_dict_offset)

In [ ]:
print('EEG event dict:')
eeg_event_dict

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='Original EEG event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
py.iplot(fig)
# print the scatterplot to an html file for easy exploration.
fig.write_html("eeg_event_times.html")

In [ ]:
if not din_str:
    print('Required EEG DIN events are missing... skipping EEG DIN check and DIN distance display')
else:
    #check that the din_str events exist in the eeg_event_dict..
    din_str = qit.din_check(eeg_event_dict, din_str)
    #get the distance between the DIN events of interest..
    din_diffs, din_diffs_time = qit.get_din_diff(eeg_events, eeg_event_dict, din_str)
    #build the figure...
    fig=px.scatter(x=din_diffs_time, y=din_diffs)
    fig.update_layout(title='Time between EEG DIN events of interest')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Inter event interval')
    # print the scatterplot to an html file for easy exploration.
    fig.write_html("eeg_din_diffs.html")

In [ ]:
if not din_str:
    print('Required EEG DIN events are missing... skipping EEG stimulus onset DIN process')
else:
    # handle task specific EEG event interpretation..
    eeg_events, eeg_stims, eeg_iti, eeg_din_offset, eeg_event_dict, new_events = qit.eeg_event_test(eeg_events, eeg_event_dict, din_str, task_name=task_id_out)

In [ ]:
# print some stim event summaries
column_values = eeg_events[:, 2]

# Get unique values and their counts
unique_values, counts = np.unique(column_values, return_counts=True)

# Print the counts of each unique value
print("Counts of each event ID value:")
for value, count in zip(unique_values, counts):
    print(f"Event ID: {value}, Count: {count}")

# Print the event labels
print("Label\tValue")
for label, value in eeg_event_dict.items():
    print(f"{label}\t{value}")

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index... including new *_d DIN events if generated
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='DIN updated EEG event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
fig.show()
fig.write_html("eeg_update_event_times.html")

In [ ]:
if not din_str:
    print('Required DIN events are missing... skipping stimulus DIN ITI display')
else:
    # peak... at the distance between *_d stim DIN events
    fig=px.scatter(x=eeg_stims[1:,0],y=eeg_iti)
    fig.update_layout(title='Stim DIN event Inter Trial Intervals (ITI)')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Stim DIN event ITI (ms)')
    fig.show()
    fig.write_html("eeg_din_iti.html")

In [ ]:
if not din_str:
    print('Required DIN events are missing... skipping stimulus DIN event offset display')
else:
    # peak... at the distance between stim events and *_d stim DIN events
    fig=px.scatter(x=eeg_stims[:,0],y=eeg_din_offset)
    fig.update_layout(title='Stim DIN offsets')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Stim DIN offset (ms)')
    fig.show()
    fig.write_html("eeg_stim_din_offset.html")

## Read the eye-tracking data

In [ ]:
    ##read the asc eye tracking data and convert it to a dataframe...
    #et_raw = mne.io.read_raw_eyelink(session_file_name_et[0])
    #et_raw.load_data()
    #data = et_raw.get_data()
    #data[np.isnan(data)] = 0
    #et_raw._data = data
    #et_raw.resample(1000, npad="auto")
    ##et_raw_fresh=et_raw.copy() #make a fresh copy for later
    #et_raw_df = et_raw.to_data_frame()
    ##get the events from the annotation structure
    #et_events, et_event_dict = mne.events_from_annotations(et_raw)
    ##et_events = mne.find_events(et_raw, min_duration=0.01, shortest_event=1, uint_cast=True)
    
    ##read the raw et asc file again this time with the blinks annotation enabled.. (this should be combined into a single read) 
    #et_raw = mne.io.read_raw_eyelink(session_file_name_et[0],create_annotations=["blinks"])
    #et_raw.load_data()
    #data = et_raw.get_data()
    #data[np.isnan(data)] = 0
    #et_raw._data = data
    #et_raw.resample(1000, npad="auto")
   
    ##interpolate the signals during blinks
    #mne.preprocessing.eyetracking.interpolate_blinks(et_raw, buffer=(0.05, 0.2), interpolate_gaze=True)


In [ ]:
et_raw, et_raw_df, et_events, et_event_dict = qit.et_read(session_file_name_et[0])

In [ ]:
if et_sync:
    # peak... at the ET channel types
    channel_types = et_raw.get_channel_types()
    print("ET Channel Types:", channel_types)
    print("ET Channel Names:", et_raw.info['ch_names'])
else:
    print("et_sync = False: not printing ET channle types")

In [ ]:
et_raw.plot(duration=20, scalings=dict(eyegaze=1e1,pupil=1e2))

## Handle the Eye-Tracking events

In [ ]:
if et_sync:
    print("ET event dict:", et_event_dict)
else:
    print("et_sync = False: not printing ET event dict")

In [ ]:
if et_sync:
    # peak... at the ET event scatter plot.. event time stamp by label index
    fig=px.scatter(x=et_events[:,0],y=et_events[:,2])
    fig.update_layout(title='Original ET event times')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Event index')
    py.iplot(fig)
    # print the scatterplot to an html file for easy exploration.
    fig.write_html("et_event_times.html")
else:
    print("et_sync = False: not plotting the original ET events")

In [ ]:
if et_sync:
    #do event cleaning..
    et_event_dict, et_events = qit.et_clean_events(et_event_dict, et_events)
    #do task specific event modifications..
    et_event_dict, et_events, et_raw_df = qit.et_task_events(et_raw_df,et_event_dict,et_events,task_id_out)
    print("updated ET event dict:", et_event_dict)
    # Extract the value for 'STIM_d' from the dictionary
    stim_d_value = et_event_dict['STIM_d']
    # Filter rows where the third column matches the 'STIM_d' value
    et_stims = et_events[et_events[:, 2] == stim_d_value]
    print('Number of stimulus onset DIN events: ' + str(len(et_stims)))
else:
    print("et_sync = False: not plotting the original ET events")

In [ ]:
if et_sync:
    # peak... at the ET event scatter plot.. event time stamp by label index
    fig=px.scatter(x=et_events[:,0],y=et_events[:,2])
    fig.update_layout(title='Updated ET event times')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Event index')
    py.iplot(fig)
    # print the scatterplot to an html file for easy exploration.
    fig.write_html("et_updated_event_times.html")
else:
    print("et_sync = False: not plotting the updated ET events")

## Examine the syncronization between the EEG and ET events

In [ ]:
if et_sync:
    #eeg_stims = eeg_stims.astype(int) * 1000/eeg_raw.info['sfreq']
    #et_stims = et_stims.astype(int) * 1000/et_raw.info['sfreq']
    # peak... at the ET event scatter plot.. event time stamp by label index
    fig=px.scatter(x=eeg_stims[:,0],y=et_stims[:,0])
    fig.update_layout(title='EEG by ET stim times')
    fig.update_xaxes(title_text='EEG stim times')
    fig.update_yaxes(title_text='ET stim times')
    py.iplot(fig)
    # print the scatterplot to an html file for easy exploration.
    fig.write_html("eeg_et_times.html")
else:
    print("et_sync = False: not plotting the EEG by ET event times")

In [ ]:
if et_sync:
    # peak... at the ET event scatter plot.. event time stamp by label index
    eeg_stims_t = eeg_stims.astype(int) * 1000/eeg_raw.info['sfreq']
    et_stims_t = et_stims.astype(int) * 1000/et_raw.info['sfreq']
    eeg_et_offset = eeg_stims_t[:,0] - et_stims_t[:,0]
    fig = px.scatter(y=eeg_et_offset)
    fig.update_layout(title='EEG ET stim event offset times')
    fig.update_xaxes(title_text='EEG ET stim times')
    fig.update_yaxes(title_text='EEG ET stim event offsets')
    fig.show()
    fig.write_html("eeg_et_sync_offsets.html")
else:
    print("et_sync = False: not plotting the EEG by ET offset times")

## Insert ET signals into the EEG raw object

In [ ]:
    #eeg_raw.load_data()
    #et_raw.load_data()

    ##add eeg_events and eeg_event_dict back in to eeg_raw as annotations..
    #eeg_event_dict_r = {value: key for key, value in eeg_event_dict.items()}
    #eeg_annots = mne.annotations_from_events(
    #    events=eeg_events,
    #    event_desc=eeg_event_dict_r,
    #    sfreq=eeg_raw.info["sfreq"],
    #    orig_time=eeg_raw.info["meas_date"],
    #)
    #eeg_raw.set_annotations(eeg_annots)

    ##n_eeg_event_dict = len(eeg_event_dict)
    ##n_et_event_dict = {key: value + n_eeg_event_dict for key, value in et_event_dict.items()}

    ##add et_events and et_event_dict back in to eeg_raw as annotations..
    #et_event_dict_r = {value: key for key, value in et_event_dict.items()}
    #et_annots = mne.annotations_from_events(
    #    events=et_events,
    #    event_desc=et_event_dict_r,
    #    sfreq=et_raw.info["sfreq"],
    #    orig_time=et_raw.info["meas_date"],
    #)
    #et_raw.set_annotations(et_annots)


In [ ]:
#eeg_event_dict

In [ ]:
#et_event_dict = {
#    (f"et_{key}" if key.startswith('DIN') else key): value
#    for key, value in et_event_dict.items()
#}

##add the eeg_event_dict row count to the values of et_event_dict
#eeg_event_count = len(eeg_event_dict)
#n_et_event_dict = {key: value + eeg_event_count for key, value in et_event_dict.items()}

#eeg_event_dict.update(n_et_event_dict)
#eeg_et_event_dict = eeg_event_dict

##add the eeg_event_dict row count to the values of et_events[:,2]    
#et_events[:, 2] += eeg_event_count
##append et_events to eeg_events
#eeg_et_events = np.vstack((eeg_events.astype(int), et_events.astype(int)))
##sort by the first column
#eeg_et_events_sorted = np.array(sorted(eeg_et_events, key=lambda x: x[0]))

#eeg_et_event_dict_r = {value: key for key, value in eeg_et_event_dict.items()}

#eeg_et_annots = mne.annotations_from_events(
#    events=eeg_et_events_sorted,
#    event_desc=eeg_et_event_dict_r,
#    sfreq=et_raw.info["sfreq"],
#   orig_time=et_raw.info["meas_date"],
#)
#eeg_raw.set_annotations(eeg_et_annots)



In [ ]:
#eeg_et_event_dict_r

In [ ]:
#eeg_et_events_sorted[:20,:]

In [ ]:
#eeg_et_annots = mne.annotations_from_events(
#    events=eeg_et_events_sorted,
#    event_desc=eeg_et_event_dict_r,
#    sfreq=et_raw.info["sfreq"],
#   orig_time=et_raw.info["meas_date"],
#)
#eeg_raw.set_annotations(eeg_et_annots)


In [ ]:
#eeg_raw.load_data()
#et_raw.load_data()

#eeg_event_dict_r = {value: key for key, value in eeg_event_dict.items()}
#eeg_annots = mne.annotations_from_events(
#    events=eeg_events,
#    event_desc=eeg_event_dict_r,
#    sfreq=eeg_raw.info["sfreq"],
#    orig_time=eeg_raw.info["meas_date"],
#)
#eeg_raw.set_annotations(eeg_annots)

#et_event_dict_r = {value: key for key, value in et_event_dict.items()}
#et_annots = mne.annotations_from_events(
#    events=et_events,
#    event_desc=et_event_dict_r,
#    sfreq=et_raw.info["sfreq"],
#   orig_time=et_raw.info["meas_date"],
#)
#et_raw.set_annotations(et_annots)

## Convert event onsets from samples to seconds
#eeg_times = eeg_stims[:, 0] / eeg_raw.info["sfreq"]
#et_times = et_stims[:, 0] / et_raw.info["sfreq"]

## Align the data
#mne.preprocessing.realign_raw(eeg_raw, et_raw, eeg_times, et_times, verbose="error")

## Add EEG channels to the eye-tracking raw object
#eeg_raw.add_channels([et_raw], force_update_info=True)
##del raw_eeg  # free up some memory

#et_events_from_annot, et_event_dict = mne.events_from_annotations(et_raw)
#eeg_events_from_annot, eeg_event_dict = mne.events_from_annotations(eeg_raw)

#et_event_dict = {
#    (f"et_{key}" if key.startswith('DIN') else key): value
#    for key, value in et_event_dict.items()
#}

#et_event_count = len(et_event_dict)
#n_eeg_event_dict = {key: value + et_event_count for key, value in eeg_event_dict.items()}

#et_event_dict.update(n_eeg_event_dict)
##eeg_et_event_dict = eeg_event_dict

#eeg_events[:, 2] += et_event_count
##append et_events to eeg_events
#et_eeg_events = np.vstack((et_events.astype(int), eeg_events.astype(int)))
##sort by the first column
#et_eeg_events_sorted = np.array(sorted(et_eeg_events, key=lambda x: x[0]))

#et_event_dict_r = {value: key for key, value in et_event_dict.items()}

#eeg_et_annots = mne.annotations_from_events(
#    events=et_eeg_events_sorted,
#    event_desc=et_event_dict_r,
#    sfreq=et_raw.info["sfreq"],
#    orig_time=et_raw.info["meas_date"],
#)
#et_raw.set_annotations(eeg_et_annots)


In [ ]:
if et_sync:
    # combine the EEG and ET recordings given the matched event times.. then create a dataframe copy..
    #eeg_et_raw = qit.eeg_et_combine(eeg_raw_fresh, et_raw_fresh, eeg_stims, et_stims)
    #eeg_et_raw = qit.eeg_et_combine(eeg_raw, et_raw, eeg_stims, et_stims)
    eeg_raw = qit.eeg_et_combine(eeg_raw, et_raw, eeg_stims, et_stims, eeg_events, eeg_event_dict, et_events, et_event_dict)
    #eeg_raw_df = eeg_et_raw.to_data_frame()
    #show channel types...
    #channel_types = eeg_raw.get_channel_types()
    #print("Channel Types:", channel_types)
    #print("Channel Names:", eeg_raw.info['ch_names'])

    # Identify the channels with specific types (eyegaze and pupil) and rename them to 'misc' (to be fixed after bids compliance updates)
    channel_types = eeg_raw.get_channel_types()
    channel_mapping = {
        ch_name: 'misc'
        for ch_name, ch_type in zip(eeg_raw.ch_names, channel_types)
        if ch_type in ['eyegaze', 'pupil']
    }

    # Update the channel types
    eeg_raw.set_channel_types(channel_mapping)

    #Update the event times since the sync
    eeg_events, eeg_event_dict = mne.events_from_annotations(eeg_raw)
    eeg_events[:,0]=eeg_events[:,0]-eeg_raw.first_samp
else:
    print('et_sync = False: skipping the integration of the ET signals into the EEG structure')

In [ ]:
# peak... at the eeg_raw properties
channel_types = eeg_raw.get_channel_types()
print("EEG Channel Types:", channel_types)
print("EEG Channel Names:", eeg_raw.info['ch_names'])
print("EEG sampling rate: ", eeg_raw.info["sfreq"])
print('EEG event dict:')
eeg_event_dict

In [ ]:
## Identify the channels with specific types (eyegaze and pupil)
#channel_types = eeg_raw.get_channel_types()
#channel_mapping = {
#    ch_name: 'misc'
#    for ch_name, ch_type in zip(eeg_raw.ch_names, channel_types)
#    if ch_type in ['eyegaze', 'pupil']
#}

## Update the channel types
#eeg_raw.set_channel_types(channel_mapping)


In [ ]:
## peak... at the eeg_raw properties
#channel_types = eeg_raw.get_channel_types()
#print("EEG Channel Types:", channel_types)
#print("EEG Channel Names:", eeg_raw.info['ch_names'])
#print("EEG sampling rate: ", eeg_raw.info["sfreq"])
#print('EEG event dict:')

In [ ]:
#eeg_event_dict

In [ ]:
#eeg_events

In [ ]:
# create the EEG event structures
#eeg_events = mne.find_events(eeg_raw, shortest_event = 1)
#eeg_event_dict = qit.get_event_dict(eeg_raw, eeg_events, event_dict_offset)
#eeg_events, eeg_event_dict = mne.events_from_annotations(eeg_raw)

In [ ]:
#eeg_events[:20,:]

In [ ]:
#eeg_event_dict

In [ ]:
# Define a few channel groups of interest and plot the data
frontal = ["E19", "E11", "E4", "E12", "E5"]
occipital = ["E61", "E62", "E78", "E67", "E72", "E77"]
din = ["DIN"]
pupil = ["pupil_left"]
x_pos = ["xpos_left"]
y_pos = ["ypos_left"]

scale_dict = dict(eeg=1e-4, eyegaze=30, pupil=30)

# picks must be numeric (not string) when passed to `raw.plot(..., order=)`
picks_idx = mne.pick_channels(eeg_raw.ch_names, din + frontal + occipital + pupil + x_pos + y_pos, ordered=True)
eeg_raw.plot(start=0,duration=4,order=picks_idx, scalings=scale_dict)

In [ ]:
#eeg_raw.first_samp

In [ ]:
#eeg_raw.times[0]

In [ ]:
#eeg_events=[]
#eeg_events

In [ ]:
#eeg_event_dict={}
#eeg_event_dict

In [ ]:
#eeg_events[:20,:]

In [ ]:
#eeg_events, eeg_event_dict = mne.events_from_annotations(eeg_raw)

In [ ]:
#eeg_events[:,0]=eeg_events[:,0]-eeg_raw.first_samp

## Write the raw structure to a BIDS directory in the project root.

In [ ]:
 # write the BIDS output files
# specify power line frequency as required by BIDS
eeg_raw.info["line_freq"] = 60
eeg_raw.info['device_info']=device_info
eeg_raw.info['device_info']['type'] = eeg_raw.info['device_info']['type'].replace(' ', '-')

def fillna(raw, fill_val=0):
    return mne.io.RawArray(np.nan_to_num(raw.get_data(), nan=fill_val), raw.info)
eeg_raw=fillna(eeg_raw,fill_val=0)

bids_path = mne_bids.BIDSPath(
    subject=subject_id_out, session=session_id, task=task_id_out, run="1", datatype="eeg", root=project_path
)

print(bids_path)
mne_bids.write_raw_bids(
    raw=eeg_raw,
    bids_path=bids_path,
    events=eeg_events,
    event_id=eeg_event_dict,
    format = "EDF",
    overwrite=True,
    allow_preload=True,
)

## Read tests...

In [ ]:
#Read test
bids_path = mne_bids.BIDSPath(
    subject="100134F1", session="01", task="VEP", run="1", datatype="eeg", root="/project/def-emayada/q1k/experimental/HSJ/"
)
raw = mne_bids.read_raw_bids(bids_path=bids_path)


In [ ]:
# peak... at the eeg_raw properties
channel_types = raw.get_channel_types()
print("EEG Channel Types:", channel_types)
print("EEG Channel Names:", raw.info['ch_names'])
print("EEG sampling rate: ", raw.info["sfreq"])

In [ ]:
# Define a few channel groups of interest and plot the data
frontal = ["E19", "E11", "E4", "E12", "E5"]
occipital = ["E61", "E62", "E78", "E67", "E72", "E77"]
din = ["DIN"]
pupil = ["pupil_left"]
x_pos = ["xpos_left"]
y_pos = ["ypos_left"]

scale_dict = dict(eeg=1e-4, eyegaze=30, pupil=30)

# picks must be numeric (not string) when passed to `raw.plot(..., order=)`
picks_idx = mne.pick_channels(raw.ch_names, din + frontal + occipital + pupil + x_pos + y_pos, ordered=True)
raw.plot(start=0,duration=4,order=picks_idx, scalings=scale_dict)

In [ ]:
## Select EEG-only channels
#eeg_raw = raw.copy().pick_types(eeg=True)

## Select all non-EEG channels
#non_eeg_raw = raw.copy().pick_types(stim=True, misc=True)


In [ ]:
#end_time = eeg_raw.times[-1]
#eeg_raw.crop(tmin=0, tmax=end_time - 2)
#non_eeg_raw.crop(tmin=0, tmax=eeg_raw.times[-1])

#eeg_raw.add_channels([non_eeg_raw], force_update_info=True)

In [ ]:
## peak... at the eeg_raw properties
#channel_types = eeg_raw.get_channel_types()
#print("EEG Channel Types:", channel_types)
#print("EEG Channel Names:", eeg_raw.info['ch_names'])
#print("EEG sampling rate: ", eeg_raw.info["sfreq"])

In [ ]:
## peak... at the eeg_raw properties
#channel_types = non_eeg_raw.get_channel_types()
#print("EEG Channel Types:", channel_types)
#print("EEG Channel Names:", non_eeg_raw.info['ch_names'])
#print("EEG sampling rate: ", non_eeg_raw.info["sfreq"])

In [ ]:
##alt read test
#raw = mne.io.read_raw_edf("../../sub-100134F1/ses-01/eeg/sub-100134F1_ses-01_task-VEP_run-1_eeg.edf",misc=['xpos_left', 'ypos_left', 'pupil_left', 'DIN', 'x_head', 'y_head', 'distance'])

## Get chan type mapping
#types = {"eyegaze": ["xpos_left", "ypos_left", "xpos_right", "ypos_right"],
#         "pupil": ["pupil_left", "pupil_right"],
#         "misc": ["x_head", "y_head", "distance"]}
#chan_type_map = {ch_name: "eeg" for ch_name in raw.ch_names if ch_name[0] == "E"}
#chan_type_map["VREF"] = "eeg"
#for type_, ch_names in types.items():
#    for ch_name in ch_names:
#        if ch_name in raw.ch_names:
#            chan_type_map[ch_name] = type_
#chan_type_map.update({ch_name: "stim" for ch_name in raw.ch_names if ch_name not in chan_type_map})

#raw.set_channel_types(chan_type_map)